In [1]:
import pandas as pd
import numpy as np

from lib.utils import get_AAPL

In [2]:
df = get_AAPL()
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [6]:
def hv(series: pd.Series, annualization_factor: int = 252) -> pd.Series:
    """Historical Volatility"""
    return series.diff().rolling(annualization_factor).std() * np.sqrt(annualization_factor)

def rv_from_log_returns(log_returns: pd.Series, window: int):
    """
    Realized volatility is defined in [Volatility Forecasting with Machine Learning
    and Intraday Commonality](https://arxiv.org/pdf/2202.08962.pdf) as:

    $$RV_{i,t}(h)=\log(\sum_{s=t-h+1}^{t}r^2_{i,s})$$
    """
    assert window > 0, "Window must be greater than 0"
    fuzz = 1e-10
    sum_of_squares = log_returns.rolling(window=window).apply(lambda x: np.sum(x**2), raw=True)
    rv = np.log(sum_of_squares + fuzz)
    return rv


def rv(series: pd.Series, window: int) -> pd.Series:
    """
    Realized volatility is defined in [Volatility Forecasting with Machine Learning
    and Intraday Commonality](https://arxiv.org/pdf/2202.08962.pdf) as:

    $$RV_{i,t}(h)=\log(\sum_{s=t-h+1}^{t}r^2_{i,s})$$
    """
    assert window > 0, "Window must be greater than 0"
    fuzz = 1e-10
    log_returns = np.log(series).diff()
    sum_of_squares = log_returns.rolling(window=window).apply(lambda x: np.sum(x**2), raw=True)
    rv = np.log(sum_of_squares + fuzz)
    return rv

In [10]:
df.loc[:, 'rv_5'] = df.loc[:, "Close"].agg(rv, window=5)

In [11]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,rv_5
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.943249,148158800,NaN
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.163826,365248800,NaN
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.622257,234428400,NaN
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.542973,219111200,NaN
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.220539,164101200,NaN
...,...,...,...,...,...,...,...,...
247,2019-12-24,71.172501,71.222504,70.730003,71.067497,69.327454,48478800,-8.202628
248,2019-12-26,71.205002,72.495003,71.175003,72.477501,70.702934,93121200,-7.332130
249,2019-12-27,72.779999,73.492500,72.029999,72.449997,70.676117,146266000,-7.333441
250,2019-12-30,72.364998,73.172501,71.305000,72.879997,71.095551,144114400,-7.287493
